In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## preprocess the data
### drop irrelevant features
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
## encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
# one hot encode 'Geography'
onehot_encoder_geo=OneHotEncoder()
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
## combine one hot encoder columns with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
## divide dataset into dependent and independent features
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [10]:
## split the data in training and testing data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [11]:
## scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [12]:
## save the encoders and scalers
with open('label_encoder_gender.pkl','wb')as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl','wb')as file:
    pickle.dump(onehot_encoder_geo,file)
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Regression problem statement

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [14]:
## build our ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),## HL1 connected with input layer
    Dense(32,activation='relu'),## HL2 
    Dense(1) ## output layer
])

In [16]:
## compile the model
model.compile(optimizer='adam',loss="mean_absolute_error",metrics=['mae'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
# set up TensorBoard
log_dir="regressionlogs/fit/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback =TensorBoard(log_dir=log_dir,histogram_freq=1)

In [20]:
## set up early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [22]:
## Training the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorboard_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 100381.1250 - mae: 100381.1250 - val_loss: 98535.9531 - val_mae: 98535.9531
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 99720.3750 - mae: 99720.3750 - val_loss: 97214.7656 - val_mae: 97214.7656
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 97429.4141 - mae: 97429.4141 - val_loss: 93857.4375 - val_mae: 93857.4375
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 92891.0234 - mae: 92891.0234 - val_loss: 88167.9844 - val_mae: 88167.9844
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 86160.0312 - mae: 86160.0312 - val_loss: 80600.1406 - val_mae: 80600.1406
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 77930.7969 - mae: 77930.7969 - val_loss: 72275.9141 - val_mae: 72275.9141
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 69505.9766 - mae: 69505.976

In [23]:
%load_ext tensorboard


In [26]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 13548), started 0:00:13 ago. (Use '!kill 13548' to kill it.)

In [27]:
## Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE :{test_mae}')

63/63 [==============================] - 0s 826us/step - loss: 50291.5352 - mae: 50291.5352
Test MAE :50291.53515625


In [29]:
model.save('regression_model.h5')

c:\Users\PRAGYA\Desktop\ANN CLASSIFICATION\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
